# Training Facebook Denoiser in Colab.

## Intro
- Author: Sung-Min, Park
- Data: Sep 12, 2025 ---
- Descriptions
  - This notebooks are written for training facebook denoiser.

---

## Step 1. Colab 환경 설정

### Step 1-1. 필수 패키지 설치

In [10]:
# 필수 패키지 설치
!pip install hydra-core==0.11.3 hydra-colorlog==0.1.4 pystoi==0.3.3 sounddevice==0.4.0 --quiet
!pip install git+https://github.com/ludlows/python-pesq#egg=pesq --quiet
!pip install julius

  Preparing metadata (setup.py) ... done


### Step 1-2. PyTorch and settings

In [2]:
import numpy as np
import torch
import torchaudio
import IPython

print(f"Numpy version: {np.__version__}")
print(f"PyTorch: {torch.__version__}")
print(f"torchaudio: {torchaudio.__version__}")

needs_restart = False

# numpy 체크 (numpy 2.x일 경우만 다운그레이드)
if np.__version__.startswith('2.'):
    print("⚠️ numpy 2.x detected. Downgrading to 1.26.4 for compatibility.")
    !pip install numpy==1.26.4 --quiet
    needs_restart = True

# GPU 확인
if torch.cuda.is_available():
    print("CUDA is available!")
    print(f"GPU: {torch.cuda.get_device_name(0)}")
    print(f"GPU 메모리: {torch.cuda.get_device_properties(0).total_memory / 1024**3:.1f}GB")
else:
    print("CUDA is unavailable, CPU only")

# 런타임을 자동으로 다시 시작해야 할 경우
if needs_restart:
    print("✅ 런타임을 자동으로 다시 시작합니다.")
    IPython.display.display(IPython.display.Javascript('google.colab.kernel.restart()'))

Numpy version: 2.0.2
PyTorch: 2.8.0+cu126
torchaudio: 2.8.0+cu126
⚠️ numpy 2.x detected. Downgrading to 1.26.4 for compatibility.
CUDA is unavailable, CPU only
✅ 런타임을 자동으로 다시 시작합니다.


<IPython.core.display.Javascript object>

### Step 1-3. Cloning Facebook Denoiser

In [3]:
# Facebook Denoiser 클론
print("Facebook Denoiser 클론 중...")
!git clone https://github.com/facebookresearch/denoiser.git

%cd denoiser

Facebook Denoiser 클론 중...
Cloning into 'denoiser'...
remote: Enumerating objects: 440, done.
remote: Total 440 (delta 0), reused 0 (delta 0), pack-reused 440 (from 1)
Receiving objects: 100% (440/440), 74.33 MiB | 19.05 MiB/s, done.
Resolving deltas: 100% (195/195), done.
/content/denoiser


## Step 2: Dataset load and preparation
- The Valentini dataset will be used.
  - The dataset must be prepared and uploaded to Google Drive in advance.
  - https://www.kaggle.com/datasets/muhmagdy/valentini-noisy

In [4]:
from google.colab import drive
drive.mount('/content/drive/')

from pathlib import Path
import os
import shutil

# 경로 설정
path_original  = "/content/drive/MyDrive/Colab Notebooks/ARMY Projects/valentini_dataset/original_dataset"
path_converted = "/content/drive/MyDrive/Colab Notebooks/ARMY Projects/valentini_dataset/converted_data"
path_denoiser  = "/content/denoiser/dataset/valentini/"

Mounted at /content/drive/


### Step 2-1. Check whether the converted data exists

In [5]:
"""
Valentini 데이터셋 변환 데이터 확인 및 복사
"""

def check_16kHz_form(file_path, default_sample_rate=16000):
    """파일이 16kHz인지 확인"""
    try:
        info = torchaudio.info(file_path)
        if info.sample_rate != default_sample_rate:
            print(f">>> ❌ - {file_path.name} - sample rate: {info.sample_rate}Hz")
            return False
        return True
    except Exception as e:
        print(f">>> ❌ - Reading Error - {file_path.name}")
        return False

def check_converted_data(converted_file_path=path_converted, check_sample=None):
    """변환된 데이터 존재 및 유효성 확인"""

    if not Path(converted_file_path).exists():
        print("❌ 변환된 데이터가 없습니다.")
        return False

    print(f"📁 경로 존재: {converted_file_path}")

    total_errors = 0
    all_folders_exist = True

    for split in ['train', 'test']:
        for type_ in ['clean', 'noisy']:
            path = f"{converted_file_path}/{split}/{type_}"

            if not Path(path).exists():
                print(f">>> ❌ {split}/{type_} 폴더가 없습니다.")
                all_folders_exist = False
                continue

            wav_files = list(Path(path).glob("*.wav"))
            count = len(wav_files)
            error_count = 0

            if count > 0:
                print(f">>> 📋 {split}/{type_}: 파일 확인 중...")

                if check_sample in ("All", "all"):
                    sample_files = wav_files
                else:
                    sample_size = min(5, count)
                    sample_ind = np.random.choice(count, size=sample_size, replace=False)
                    sample_files = [wav_files[i] for i in sample_ind]

                for sample_file in sample_files:
                    if not check_16kHz_form(sample_file):
                        error_count += 1

                total_errors += error_count
                print(f">>> ✅ {split}/{type_}: 총 {count}개, 16kHz 파일 {count - error_count}개")

                if error_count > 0:
                    print(f">>> ❌ {split}/{type_}: Error가 {error_count}개 있습니다.")
            else:
                print(f">>> ⚠️ {split}/{type_}: 파일이 없습니다.")
                all_folders_exist = False

    if not all_folders_exist:
        print("❌ 폴더 구조가 불완전합니다.")
        return False

    if total_errors > 0:
        print(f"❌ 총 {total_errors}개의 파일이 16kHz가 아닙니다.")
        return False

    print("✅ 모든 검증 통과! 16kHz 데이터 준비 완료")
    return True

def copy_to_denoiser(denoiser_dataset_path=path_denoiser,
                     converted_file_path=path_converted):
    """변환된 데이터를 denoiser 경로로 복사"""

    print(f"\n📋 Denoiser 경로로 복사: {denoiser_dataset_path}")

    try:
        if Path(denoiser_dataset_path).exists():
            shutil.rmtree(denoiser_dataset_path)
        os.makedirs(denoiser_dataset_path)

        for item in Path(converted_file_path).iterdir():
            if item.is_dir():
                shutil.copytree(item, Path(denoiser_dataset_path) / item.name)

        print("✅ Denoiser 복사 완료!")
        return True

    except Exception as e:
        print(f"❌ 복사 중 오류: {e}")
        return False

if __name__ == "__main__":
    print("🔍 Valentini 데이터 확인 시작")
    print("=" * 50)

    if check_converted_data(converted_file_path=path_converted,
                            check_sample="All"):

        print("\n⚡ 기존 변환 데이터 재사용!")

        if copy_to_denoiser():
            print("🎯 데이터 준비 완료! 훈련 시작 가능!")
        else:
            print("❌ 복사 실패!")
    else:
        print("\n🔄 변환이 필요합니다.")
        print("💡 변환 코드를 실행하세요.")

🔍 Valentini 데이터 확인 시작
📁 경로 존재: /content/drive/MyDrive/Colab Notebooks/ARMY Projects/valentini_dataset/converted_data
>>> 📋 train/clean: 파일 확인 중...


/tmp/ipython-input-2322691637.py:8: UserWarning: torchaudio._backend.utils.info has been deprecated. This deprecation is part of a large refactoring effort to transition TorchAudio into a maintenance phase. The decoding and encoding capabilities of PyTorch for both audio and video are being consolidated into TorchCodec. Please see https://github.com/pytorch/audio/issues/3902 for more information. It will be removed from the 2.9 release. 
  info = torchaudio.info(file_path)
/usr/local/lib/python3.12/dist-packages/torchaudio/_backend/ffmpeg.py:20: UserWarning: torio.io._streaming_media_decoder.StreamingMediaDecoder has been deprecated. This deprecation is part of a large refactoring effort to transition TorchAudio into a maintenance phase. The decoding and encoding capabilities of PyTorch for both audio and video are being consolidated into TorchCodec. Please see https://github.com/pytorch/audio/issues/3902 for more information. It will be removed from the 2.9 release. 
  s = torchaudio.

>>> ✅ train/clean: 총 11572개, 16kHz 파일 11572개
>>> 📋 train/noisy: 파일 확인 중...
>>> ✅ train/noisy: 총 11572개, 16kHz 파일 11572개
>>> 📋 test/clean: 파일 확인 중...
>>> ✅ test/clean: 총 824개, 16kHz 파일 824개
>>> 📋 test/noisy: 파일 확인 중...
>>> ✅ test/noisy: 총 824개, 16kHz 파일 824개
✅ 모든 검증 통과! 16kHz 데이터 준비 완료

⚡ 기존 변환 데이터 재사용!

📋 Denoiser 경로로 복사: /content/denoiser/dataset/valentini/
✅ Denoiser 복사 완료!
🎯 데이터 준비 완료! 훈련 시작 가능!


### Step 2-2. Convert data

In [ ]:
def resample_and_save(source_file, target_dir, default_resample=16000):
    """파일을 16kHz로 리샘플링하여 저장"""
    try:
        # 오디오 로드
        waveform, orig_sr = torchaudio.load(source_file)

        # 16kHz로 리샘플링 (필요한 경우)
        if orig_sr != default_resample:
            resampler = torchaudio.transforms.Resample(orig_sr, default_resample)
            resampled = resampler(waveform)
        else:
            resampled = waveform

        # 저장 경로 생성
        output_path = target_dir / source_file.name
        torchaudio.save(str(output_path), resampled, default_resample)

        return True

    except Exception as e:
        print(f"❌ Resampling Error {source_file.name}: {e}")
        return False


def convert_and_backup_valentini(folder_mapping, default_resample=16000):
    """원본 데이터를 16kHz로 변환하고 Google Drive에 저장"""

    print("🔄 원본 데이터 변환 시작...")

    # Google Drive 저장 경로 생성
    os.makedirs(path_converted, exist_ok=True)

    total_converted = 0
    total_errors = 0

    for source_folder, target_folder in folder_mapping.items():
        source_dir = Path(path_original) / source_folder
        target_dir = Path(path_converted) / target_folder

        if not source_dir.exists():
            print(f"⚠️  {source_dir} 폴더가 없습니다. 건너뜀.")
            continue

        os.makedirs(target_dir, exist_ok=True)

        # 원본 파일 목록
        source_files = list(source_dir.glob("*.wav"))
        print(f"📂 {source_folder}: {len(source_files)}개 파일")

        converted = 0
        errors = 0

        for file in tqdm(source_files,
                        desc=f"🔄 {source_folder}",
                        unit="files", leave=True):

            if resample_and_save(file, target_dir, default_resample):
                converted += 1
            else:
                errors += 1

        print(f"✅ {source_folder}: {converted}개 성공, {errors}개 실패")
        total_converted += converted
        total_errors += errors

    # 결과 출력
    if total_converted > 0:
        print(f"\n💾 Google Drive 저장 완료! ({total_converted}개 파일)")

        # 저장 검증
        saved_files = 0
        for split in ['train', 'test']:
            for type_ in ['clean', 'noisy']:
                path = f"{path_converted}/{split}/{type_}"
                if Path(path).exists():
                    count = len(list(Path(path).glob("*.wav")))
                    saved_files += count
                    print(f">>> ✅ {split}/{type_}: {count}개 파일")

        print(f"✅ 저장 검증: {saved_files}개 파일")
        return True
    else:
        print("❌ 변환된 파일이 없습니다.")
        return False


# 🚀 메인 실행
if __name__ == "__main__":
    print("="*60)
    print("🎵 Valentini 데이터셋 16kHz 변환")
    print("="*60)

    folder_mapping = {
        "train_clean_28spk_wav": "train/clean",
        "train_noisy_28spk_wav": "train/noisy",
        "test_clean_wav": "test/clean",
        "test_noisy_wav": "test/noisy"
    }

    if not Path(path_original).exists():
        print(f"❌ 원본 데이터 경로가 없습니다: {path_original}")
    else:
        success = convert_and_backup_valentini(folder_mapping)
        if success:
            print("🎉 변환 완료!")
        else:
            print("❌ 변환 실패!")

🎵 Valentini 데이터셋 16kHz 변환 관리
💡 한 번 변환하면 Google Drive에 저장되어 재사용됩니다!
🔄 원본 데이터 변환 시작...
📂 train_clean_28spk_wav: 11572개 파일


🔄 train_clean_28spk_wav:   0%|          | 0/11572 [00:00<?, ?files/s]/usr/local/lib/python3.12/dist-packages/torchaudio/_backend/utils.py:213: UserWarning: In 2.9, this function's implementation will be changed to use torchaudio.load_with_torchcodec` under the hood. Some parameters like ``normalize``, ``format``, ``buffer_size``, and ``backend`` will be ignored. We recommend that you port your code to rely directly on TorchCodec's decoder instead: https://docs.pytorch.org/torchcodec/stable/generated/torchcodec.decoders.AudioDecoder.html#torchcodec.decoders.AudioDecoder.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/torchaudio/_backend/ffmpeg.py:88: UserWarning: torio.io._streaming_media_decoder.StreamingMediaDecoder has been deprecated. This deprecation is part of a large refactoring effort to transition TorchAudio into a maintenance phase. The decoding and encoding capabilities of PyTorch for both audio and video are being consolidated into TorchCodec. Please see https://gi

✅ train_clean_28spk_wav: 11572개 성공, 0개 실패
📂 train_noisy_28spk_wav: 11572개 파일


🔄 train_noisy_28spk_wav: 100%|██████████| 11572/11572 [30:06<00:00,  6.41files/s]


✅ train_noisy_28spk_wav: 11572개 성공, 0개 실패
📂 test_clean_wav: 824개 파일


🔄 test_clean_wav: 100%|██████████| 824/824 [02:15<00:00,  6.09files/s]


✅ test_clean_wav: 824개 성공, 0개 실패
📂 test_noisy_wav: 824개 파일


🔄 test_noisy_wav: 100%|██████████| 824/824 [02:10<00:00,  6.33files/s]


✅ test_noisy_wav: 824개 성공, 0개 실패

💾 Google Drive 저장 완료! (24792개 파일)
✅ 저장 검증: 24792개 파일

🎯 변환 완료! Denoiser로 복사 중...

📋 Denoiser 경로로 복사: /content/denoiser/dataset/valentini/


## Step 3: Json 메타 데이터 생성

In [13]:
import json
import os

In [14]:
# !pwd  # /content/denoiser
# 데이터 폴더 구조 확인
!ls -la "/content/denoiser/dataset/"
!ls -la "/content/denoiser/dataset/valentini"
!ls -la "/content/denoiser/egs"

total 16
drwxr-xr-x 4 root root 4096 Sep 22 01:12 .
drwxr-xr-x 9 root root 4096 Sep 22 01:28 ..
drwxr-xr-x 4 root root 4096 Sep 22 00:51 debug
drwxr-xr-x 4 root root 4096 Sep 22 01:22 valentini
total 16
drwxr-xr-x 4 root root 4096 Sep 22 01:22 .
drwxr-xr-x 4 root root 4096 Sep 22 01:12 ..
drwx------ 4 root root 4096 Sep 17 05:44 test
drwx------ 4 root root 4096 Sep 17 04:54 train
total 12
drwxr-xr-x 3 root root 4096 Sep 22 01:28 .
drwxr-xr-x 9 root root 4096 Sep 22 01:28 ..
drwxr-xr-x 4 root root 4096 Sep 22 01:28 valentini


In [16]:
!pip install julius

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.6/59.6 kB 2.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for julius: filename=julius-0.2.7-py3-none-any.whl size=21870 sha256=846aa86c974d059d6f460ca4beb1846e2ac5adb01b27ebbb712a55e1acce3c6c
  Stored in directory: /root/.cache/pip/wheels/de/c1/ca/544dafe48401e8e2e17064dfe465a390fca9e8720ffa12e744
Successfully built julius


In [17]:
# 메타데이터 폴더 생성
!mkdir -p egs/valentini/{tr,tt}

print("JSON 메타데이터 파일 생성 중...")

# 훈련 데이터 JSON 생성
print("📁 훈련 데이터 JSON 생성...")
!python -m denoiser.audio dataset/valentini/train/clean > egs/valentini/tr/clean.json
!python -m denoiser.audio dataset/valentini/train/noisy > egs/valentini/tr/noisy.json

# 테스트 데이터 JSON 생성
print("📁 테스트 데이터 JSON 생성...")
!python -m denoiser.audio dataset/valentini/test/clean > egs/valentini/tt/clean.json
!python -m denoiser.audio dataset/valentini/test/noisy > egs/valentini/tt/noisy.json

# 생성된 JSON 파일 검증
print("\n📊 생성된 JSON 파일 통계:")
json_files = [
    "egs/valentini/tr/clean.json",
    "egs/valentini/tr/noisy.json",
    "egs/valentini/tt/clean.json",
    "egs/valentini/tt/noisy.json"
]

for json_file in json_files:
    if os.path.exists(json_file):
        with open(json_file, 'r') as f:
            data = json.load(f)
        print(f"  {json_file}: {len(data)}개 파일")

        # 첫 번째 항목 샘플 출력
        if data and json_file == "egs/valentini/tr/clean.json":
            print(f"    샘플: {data[0]}")
    else:
        print(f"  ❌ {json_file}: 파일이 없습니다")

print("\n✅ JSON 메타데이터 생성 완료!")

JSON 메타데이터 파일 생성 중...
📁 훈련 데이터 JSON 생성...
/content/denoiser/denoiser/audio.py:24: UserWarning: torchaudio._backend.utils.info has been deprecated. This deprecation is part of a large refactoring effort to transition TorchAudio into a maintenance phase. The decoding and encoding capabilities of PyTorch for both audio and video are being consolidated into TorchCodec. Please see https://github.com/pytorch/audio/issues/3902 for more information. It will be removed from the 2.9 release. 
  info = torchaudio.info(path)
/usr/local/lib/python3.12/dist-packages/torchaudio/_backend/ffmpeg.py:20: UserWarning: torio.io._streaming_media_decoder.StreamingMediaDecoder has been deprecated. This deprecation is part of a large refactoring effort to transition TorchAudio into a maintenance phase. The decoding and encoding capabilities of PyTorch for both audio and video are being consolidated into TorchCodec. Please see https://github.com/pytorch/audio/issues/3902 for more information. It will be removed

In [18]:
# Step 5: 라즈베리파이 5 최적화 설정 파일 생성

import os

# 1. Valentini Colab 데이터셋 설정
valentini_config = """# Valentini 데이터셋 설정 (Colab)
dset:
  train: egs/valentini/tr
  valid:
  test: egs/valentini/tt
  noisy_json: egs/valentini/tt/noisy.json
  noisy_dir:
  matching: sort
eval_every: 10
pesq: 1
"""

os.makedirs('conf/dset', exist_ok=True)
with open('conf/dset/valentini_colab.yaml', 'w') as f:
    f.write(valentini_config)

print("✅ Valentini Colab 설정 파일 생성 완료")

# 2. 라즈베리파이 5 최적화 모델 설정 (선택사항)
rpi5_optimal_config = """# 라즈베리파이 5 최적화 모델 설정
defaults:
  - dset: valentini_colab
  - hydra/job_logging: colorlog
  - hydra/hydra_logging: colorlog

# 기본 설정
sample_rate: 16000
segment: 4
stride: 1
pad: true

# 데이터 증강 (Valentini용)
remix: true
bandmask: 0.2
shift: 8000
shift_same: true
revecho: 0

# 훈련 설정
epochs: 150
batch_size: 32
lr: 3e-4
optim: adam
beta2: 0.999
loss: l1

# STFT 손실 (품질 향상)
stft_loss: true
stft_sc_factor: 0.1
stft_mag_factor: 0.1

# 평가 설정
eval_every: 10
pesq: true

# 라즈베리파이 5 최적화 모델 아키텍처 (RTF=0.834 달성)
demucs:
  chin: 1
  chout: 1
  hidden: 32          # 48→32 (핵심 최적화)
  depth: 4            # 5→4 감소
  kernel_size: 8      # 유지 (변경 금지)
  stride: 4           # 유지 (변경 금지)
  causal: true        # 실시간 필수
  glu: false          # true→false (파라미터 50% 감소)
  resample: 2         # 4→2 (2.5배 성능 향상)
  growth: 1.5         # 2.0→1.5 경량화
  max_hidden: 128     # hidden*4로 제한
  normalize: true     # 유지
  rescale: 0.1        # 유지

# 시스템 설정
device: cuda
num_workers: 2
checkpoint: true
"""

with open('conf/rpi5_optimal.yaml', 'w') as f:
    f.write(rpi5_optimal_config)

print("✅ 라즈베리파이 5 최적화 설정 파일 생성 완료")

# 생성된 설정 파일 확인
print("\n📁 생성된 설정 파일:")
print("  - conf/dset/valentini_colab.yaml")
print("  - conf/rpi5_optimal.yaml")

# 설정 내용 미리보기
print("\n📋 주요 모델 파라미터 (라즈베리파이 5 최적화):")
print("  - hidden: 32 (경량화)")
print("  - depth: 4 (성능 균형)")
print("  - glu: false (파라미터 50% 감소)")
print("  - resample: 2 (2.5배 성능 향상)")
print("  - growth: 1.5 (메모리 최적화)")
print("  - 예상 RTF: 0.834 (라즈베리파이 5 기준)")

✅ Valentini Colab 설정 파일 생성 완료
✅ 라즈베리파이 5 최적화 설정 파일 생성 완료

📁 생성된 설정 파일:
  - conf/dset/valentini_colab.yaml
  - conf/rpi5_optimal.yaml

📋 주요 모델 파라미터 (라즈베리파이 5 최적화):
  - hidden: 32 (경량화)
  - depth: 4 (성능 균형)
  - glu: false (파라미터 50% 감소)
  - resample: 2 (2.5배 성능 향상)
  - growth: 1.5 (메모리 최적화)
  - 예상 RTF: 0.834 (라즈베리파이 5 기준)


In [9]:
# 생성된 파일 확인
!ls -la conf/rpi5_optimal.yaml
!ls -la conf/dset/valentini_colab.yaml

# 설정 내용 간단 확인
!head -20 conf/rpi5_optimal.yaml

-rw-r--r-- 1 root root 1192 Sep 18 05:16 conf/rpi5_optimal.yaml
-rw-r--r-- 1 root root 203 Sep 18 05:16 conf/dset/valentini_colab.yaml
# 라즈베리파이 5 최적화 모델 설정
defaults:
  - dset: valentini_colab
  - hydra/job_logging: colorlog
  - hydra/hydra_logging: colorlog

# 기본 설정
sample_rate: 16000
segment: 4
stride: 1
pad: true

# 데이터 증강 (Valentini용)
remix: true
bandmask: 0.2
shift: 8000
shift_same: true
revecho: 0

# 훈련 설정


## Step 4. Training

🚀 라즈베리파이 5 최적화 모델 훈련 시작

모델 설정:
  - Architecture: Light-32-Depth4
  - Dataset: Valentini (16kHz)
  - Target RTF: 0.834 (Raspberry Pi 5)
  - Epochs: 150
  - Expected time: 4-8 hours (GPU)

In [5]:
!cd /content/denoiser && ls

"""
!cd /content/denoiser && ls
CODE_OF_CONDUCT.md  img			    README.md
conf		    launch_dns.sh	    requirements_cuda.txt
CONTRIBUTING.md     launch_valentini_nc.sh  requirements.txt
dataset		    launch_valentini.sh     setup.cfg
denoiser	    LICENSE		    setup.py
egs		    make_debug.sh	    train.py
hubconf.py	    MANIFEST.in
"""

/bin/bash: line 1: cd: /content/denoiser: No such file or directory


'\n!cd /content/denoiser && ls\nCODE_OF_CONDUCT.md  img\t\t\t    README.md\nconf\t\t    launch_dns.sh\t    requirements_cuda.txt\nCONTRIBUTING.md     launch_valentini_nc.sh  requirements.txt\ndataset\t\t    launch_valentini.sh     setup.cfg\ndenoiser\t    LICENSE\t\t    setup.py\negs\t\t    make_debug.sh\t    train.py\nhubconf.py\t    MANIFEST.in\n'

In [2]:
!cd /content/denoiser && python train.py \
  hydra.job.chdir=True \
  dset.train=egs/valentini/tr \
  dset.test=egs/valentini/tt \
  dset.noisy_json=egs/valentini/tt/noisy.json \
  dset.matching=sort \
  demucs.hidden=32 \
  demucs.depth=4 \
  demucs.glu=false \
  demucs.resample=2 \
  demucs.growth=1.5 \
  demucs.max_hidden=128 \
  epochs=150 \
  batch_size=32 \
  stft_loss=true \
  remix=true \
  bandmask=0.2 \
  shift=8000 \
  shift_same=true

/content/denoiser/train.py:102: UserWarning: 
The version_base parameter is not specified.
Please specify a compatability version level, or None.
Will assume defaults for version 1.1
  @hydra.main(config_path="conf/config.yaml")
Traceback (most recent call last):
  File "/content/denoiser/train.py", line 113, in <module>
    main()
  File "/usr/local/lib/python3.12/dist-packages/hydra/main.py", line 94, in decorated_main
    _run_hydra(
  File "/usr/local/lib/python3.12/dist-packages/hydra/_internal/utils.py", line 335, in _run_hydra
    validate_config_path(config_path)
  File "/usr/local/lib/python3.12/dist-packages/hydra/core/utils.py", line 293, in validate_config_path
    raise ValueError(msg)
ValueError: Using config_path to specify the config name is not supported, specify the config name via config_name.
See https://hydra.cc/docs/1.2/upgrades/0.11_to_1.0/config_path_changes



In [4]:
# 파일 위치 재확인
!ls /content/denoiser/train.py

# 올바른 경로로 수정
!cd /content/denoiser && grep -n "@hydra.main" train.py

# 수정 실행
!cd /content/denoiser && sed -i '102s/@hydra.main(config_path="conf\/config.yaml")/@hydra.main(version_base=None, config_path="conf", config_name="config")/' train.py

# 수정 결과 확인
!cd /content/denoiser && grep -n "@hydra.main" train.py

/content/denoiser/train.py
102:@hydra.main(config_path="conf/config.yaml")
102:@hydra.main(version_base=None, config_path="conf", config_name="config")


In [10]:
def check_progress():
    os.chdir('/content/denoiser')

    # 최신 실험 폴더 찾기
    dirs = glob.glob("outputs/exp_*")
    if not dirs:
        print("❌ 훈련 폴더 없음")
        return

    latest = max(dirs)
    history_file = f"{latest}/history.json"

    if not os.path.exists(history_file):
        print("⏳ history.json 대기 중...")
        return

    # 히스토리 읽기
    with open(history_file, 'r') as f:
        history = json.load(f)

    if not history:
        print("⏳ 데이터 대기 중...")
        return

    # 최신 결과 출력
    latest_result = history[-1]
    epoch = len(history)

    print(f"📈 에포크: {epoch}/150")
    print(f"📉 Train Loss: {latest_result.get('train', 'N/A'):.5f}")
    print(f"📉 Valid Loss: {latest_result.get('valid', 'N/A'):.5f}")

    # PESQ/STOI (있으면 출력)
    if 'pesq' in latest_result:
        pesq = latest_result['pesq']
        status = "🟢" if pesq > 2.5 else "🟡" if pesq > 2.0 else "🔴"
        print(f"🎵 PESQ: {pesq:.3f} {status}")

    if 'stoi' in latest_result:
        stoi = latest_result['stoi']
        status = "🟢" if stoi > 0.85 else "🟡" if stoi > 0.75 else "🔴"
        print(f"🔊 STOI: {stoi:.3f} {status}")



In [8]:
# debug 데이터셋 생성 (작은 데이터로 테스트)
!cd /content/denoiser && ./make_debug.sh

# 기본 설정으로 실행 (에러 없이 작동)
!cd /content/denoiser && python train.py num_workers=0 device=cuda epochs=5 demucs.hidden=32 demucs.depth=4 demucs.glu=false demucs.resample=2

/content/denoiser/denoiser/audio.py:24: UserWarning: torchaudio._backend.utils.info has been deprecated. This deprecation is part of a large refactoring effort to transition TorchAudio into a maintenance phase. The decoding and encoding capabilities of PyTorch for both audio and video are being consolidated into TorchCodec. Please see https://github.com/pytorch/audio/issues/3902 for more information. It will be removed from the 2.9 release. 
  info = torchaudio.info(path)
/usr/local/lib/python3.12/dist-packages/torchaudio/_backend/ffmpeg.py:20: UserWarning: torio.io._streaming_media_decoder.StreamingMediaDecoder has been deprecated. This deprecation is part of a large refactoring effort to transition TorchAudio into a maintenance phase. The decoding and encoding capabilities of PyTorch for both audio and video are being consolidated into TorchCodec. Please see https://github.com/pytorch/audio/issues/3902 for more information. It will be removed from the 2.9 release. 
  s = torchaudio.i